In [1]:
import pdb
import numpy as np
import pandas as pd
import torch.nn as nn

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


from ML_for_WDN.data_utils import clean_dataframes, load_data

from ML_for_WDN.models import (
    SupervisedReconstructionLeakDetector,
    SupervisedLatentLeakDetector,
)

# set working directory to root directory
%cd ..


DATA_FILES = [
    'data/data_no_leak.xlsx',
    'data/data_leak_1.xlsx',
    'data/data_leak_2.xlsx',
    'data/data_leak_3.xlsx',
]

train_fraction = 0.8
test_fraction = 0.2

columns_to_use = [
    'FM01_flow', 'FM02_head', 'FM03_flow', 'FM05_flow', 'FM06_flow', 'FM08_flow', 'FM09_flow', 'FM11_flow', 'FM13_flow',
    'FM01_head', 'FM02_flow', 'FM03_head', 'FM05_head', 'FM06_head', 'FM08_head', 'FM09_head', 'FM11_head', 'FM13_head',
]

# Train data
dataframes = []
for data_file in DATA_FILES:
    df = load_data(data_file)
    dataframes.append(df)

dataframes = clean_dataframes(
    dataframes,
    columns_to_use=columns_to_use,
)

num_train_pr_dataframe = []
num_test_pr_dataframe = []
train_dataframes = []
test_dataframes = []
for df in dataframes:
    num_train_pr_dataframe.append(int(train_fraction*df.shape[0]))
    train_dataframes.append(df.iloc[:int(train_fraction*df.shape[0]), :])

    num_test_pr_dataframe.append(int(test_fraction*df.shape[0]))
    test_dataframes.append(df.iloc[-int(test_fraction*df.shape[0]):, :])

X_train = pd.concat(train_dataframes, ignore_index=True).values
X_test = pd.concat(test_dataframes, ignore_index=True).values

# Create targets
y_train = []
y_test = []
for i in range(len(num_train_pr_dataframe)):
    y_train += [i]*num_train_pr_dataframe[i]
    y_test += [i]*num_test_pr_dataframe[i]

y_train = np.array(y_train)
y_test = np.array(y_test)

/export/scratch1/ntm/nikolaj_workstation_data/PhD/ML-for-WDN


In [2]:
logistic_regression_args = {
    'penalty': 'l2',
    'C': 1e-2,
    'solver': 'lbfgs',
    'max_iter': 1000,
}
    
svc_args = {
    'C': 1e-2,
    'kernel': 'rbf',
    'gamma': 1e-2,
}

rf_args = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
}

reconstruction_WAE_args = {
    'encoder_args': {
        'hidden_dims': [12, 10, 8],
        'latent_dim': 4,
    },
    'decoder_args': {
        'latent_dim': 4,
        'hidden_dims': [8, 10, 12],
        'num_pars': 4
    },
    'NN_train_args': {
        'epochs': 5,
        'batch_size': 2048,
        'lr': 5e-3,
        'weight_decay': 1e-10,
        'loss_fn': nn.MSELoss(),
    },
    'device': 'cpu'
}

latent_WAE_args = {
    'encoder_args': {
        'hidden_dims': [12, 10, 8],
        'latent_dim': 4,
    },
    'decoder_args': {
        'latent_dim': 4,
        'hidden_dims': [8, 10, 12],
    },
    'NN_train_args': {
        'epochs': 5,
        'batch_size': 2048,
        'lr': 5e-3,
        'weight_decay': 1e-10,
        'loss_fn': nn.MSELoss(),
    },
    'device': 'cpu'
}



In [3]:
model_list = [
    #LogisticRegression(**logistic_regression_args),
    #SVC(**svc_args),
    #RandomForestClassifier(**rf_args),
    #SupervisedReconstructionLeakDetector(**reconstruction_WAE_args),
    SupervisedLatentLeakDetector(**latent_WAE_args),
]

for model in model_list:
    pipeline = Pipeline([
        ('scaler',  MinMaxScaler()),
        ('model', model),
    ])

    pipeline.fit(
        X=X_train,
        y=y_train,
    )

    preds = pipeline.predict(
        X=X_test,
    )
    
    cm = confusion_matrix(y_test, preds)
    print(f'{model}: Accuracy: {accuracy_score(y_test, preds):0.3f}')
    

########## Training stage 1 ##########


Training autoencoder without leak data


Autoencoder architecture:
- Latent dimension: 4
- Encoder hidden dimensions: [12, 10, 8]
- Decoder hidden dimensions: [8, 10, 12]




  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.5271 | Latent: 0.4733: 100%|██████████| 5/5 [00:37<00:00,  7.42s/it]




Autoencoder training complete


########## Training stage 2 ##########


Training anomaly detector using autoencoder


SupervisedLatentLeakDetector(NN_train_args={'batch_size': 2048, 'epochs': 5,
                                            'loss_fn': MSELoss(), 'lr': 0.005,
                                            'weight_decay': 1e-10},
                             decoder_args={'hidden_dims': [8, 10, 12],
                                           'latent_dim': 4, 'output_dim': 18},
                             device='cpu',
                             encoder_args={'hidden_dims': [12, 10, 8],
                                           'input_dim': 18, 'latent_dim': 4}): Accuracy: 0.382
